In [1]:
from langchain.agents import AgentType
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

MODEL_NAME = "google/flan-t5-large"


llm_transformers_pipeline = pipeline(
    "text2text-generation",
    model=AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME),
    tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME),
    max_new_tokens=200,
)

llm = HuggingFacePipeline(pipeline=llm_transformers_pipeline)

Device set to use cpu


In [4]:
response =  llm.invoke("who is Bill Gates?")
print(response)

computer scientist


## Function

In AI, a **tool** will call a basic **function** or capability that can be called on to perform a specific task. Think of it like a single item in a toolbox: just like a hammer, screwdriver, or wrench, an AI toolbox is full of specific functions designed to solve problems or get things done.

When building tools for tool calling, there are a few key principles to keep in mind:

1. **Clear purpose**: Make sure the tool has a well-defined job.
2. **Standardized input**: The tool should accept input in a predictable, structured format so it’s easy to use.
3. **Consistent output**: Always return results in a format that’s easy to process or integrate with other systems.
4. **Comprehensive documentation**: Your tool should include clear, simple documentation that explains what it does, how to use it, and any quirks or limitations.

Remember, documentation isn’t just for other developers—it’s also for the language model (LLM) to understand the tool’s purpose and how to use it effectively.

For this example, you’ll start with a simple tool to add numbers. It’ll check off most of the basic requirements, but one key limitation is that it doesn’t handle **basic error cases**, like ignoring non-numeric input. Improving error handling will make the tool much more robust and ready for real-world use.

## Improved tool return types with Python typing

When creating tools, you must accurately specify their return values. This helps the agent understand and handle different possible outputs.



In [5]:
'''def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string.

    Parameters:
    - inputs (str): 
    string, it should contain numbers that can be extracted and summed.

    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.

    Example Input (Dictionary):
    {"numbers": [10, 20, 30]}

    Example Input (String):
    "Add the numbers 10, 20, and 30."

    Example Output:
    {"result": 60}
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]

    
    result = sum(numbers)
    return {"result": result}


from langchain.agents import Tool
add_tool = Tool(
    name="AddTool",
    func=add_numbers,
    description="Adds a list of numbers provided in the input dictionary or extracts numbers from a string."
)

'''

# Option 2: using @tool decorator (RECOMMENDED)

from langchain_core.tools import tool
import re
from typing import Dict, Union

@tool
def add_numbers(inputs: str) -> str:
    """
    Adds a list of numbers provided in the input string.
    Parameters:
    - inputs (str): string containing numbers that can be extracted and summed.
    Returns:
    - str: The sum of the numbers found in the input string.
    Example Input: "Add the numbers 10, 20, and 30."
    Example Output: "60"
    """
    # Use regular expressions to extract all numbers from the input
    numbers_found = re.findall(r'\d+', inputs)
    if not numbers_found:
        return "No numbers found in the input."
    try:
        numbers = [int(num) for num in numbers_found]
        result = sum(numbers)
        return str(result)
    except Exception as e:
        return f"Error during summation: {str(e)}"



print(add_numbers.name, add_numbers.description, add_numbers.args)
print( add_numbers.invoke("Sum the numbers 10, 20, and 30."))

add_numbers Adds a list of numbers provided in the input string.
Parameters:
- inputs (str): string containing numbers that can be extracted and summed.
Returns:
- str: The sum of the numbers found in the input string.
Example Input: "Add the numbers 10, 20, and 30."
Example Output: "60" {'inputs': {'title': 'Inputs', 'type': 'string'}}
60


In [6]:
@tool
def search_wikipedia(term: str) -> str:
    """
    Searches Wikipedia for the given query and returns the summary.
    
    Parameters:
    - term (str): The search term to look up on Wikipedia.
    
    Returns:
    - str: Wikipedia summary of the search term.
    
    Example Input: "Python programming language"
    Example Output: "Python is an interpreted, high-level, general-purpose programming language."
    """
    import wikipedia
    try:
        summary = wikipedia.summary(term, sentences=3)
        return summary
    except Exception as e:
        return f"Error fetching summary from Wikipedia: {str(e)}"

#print(search_wikipedia.invoke("war"))

In [7]:
tools = [add_numbers, search_wikipedia]
tools

[StructuredTool(name='add_numbers', description='Adds a list of numbers provided in the input string.\nParameters:\n- inputs (str): string containing numbers that can be extracted and summed.\nReturns:\n- str: The sum of the numbers found in the input string.\nExample Input: "Add the numbers 10, 20, and 30."\nExample Output: "60"', args_schema=<class 'langchain_core.utils.pydantic.add_numbers'>, func=<function add_numbers at 0x0000014D20BA9120>),
 StructuredTool(name='search_wikipedia', description='Searches Wikipedia for the given query and returns the summary.\n\nParameters:\n- term (str): The search term to look up on Wikipedia.\n\nReturns:\n- str: Wikipedia summary of the search term.\n\nExample Input: "Python programming language"\nExample Output: "Python is an interpreted, high-level, general-purpose programming language."', args_schema=<class 'langchain_core.utils.pydantic.search_wikipedia'>, func=<function search_wikipedia at 0x0000014D20BA93A0>)]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,

)

### `initialize_agent`

When you set up an agent, you're connecting tools and an LLM to work together seamlessly. The agent uses the LLM to understand what needs to be done and decides which tool to use based on the task. Here's a simple overview of the key parts:


#### **Relationship between agent and LLM**
- The **agent** acts as the decision-maker, figuring out which tools to use and when.
- The **LLM** is the reasoning engine. It:
  - Interprets the user's input.
  - Helps the agent make decisions.
  - Generates a response based on the output of the tools.

Think of the agent as the manager assigning tasks and the LLM as the brain solving problems or delegating work.

---

#### **Key parameters of `initialize_agent`**

1. **`tools`**- see above 

2.  **`llm`** see above 

3. **`agent`**:
   - Specifies the reasoning framework for the agent.
   - `"zero-shot-react-description"` enables:
     - **Zero-shot reasoning**: The agent can solve tasks it hasn't seen before by thinking through the problem step by step.
     - **React framework**: A logical loop of:
       - **Reason** → Think about the task.
       - **Act** → Use a tool to perform an action.
       - **Observe** → Check the tool's output.
       - **Plan** → Decide what to do next.

4. **`verbose`**:
   - If `True`, it prints detailed logs of the agent’s thought process.
   - Useful for debugging or understanding how the agent makes decisions.


In [37]:
from langchain.agents import initialize_agent

agent = initialize_agent([search_wikipedia], llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)

In [36]:
agent.invoke({"input": "What is the sum of 10, 20, and 30? Also, can you tell me about the Python programming language?"})



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'"summary"', '"result"'}